In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)
response = llm.invoke("How do I know my girlfriend is not a goat?")
print(response)

content="That's an interesting question.  To determine if your girlfriend is indeed a human and not a goat, here are a few things you can consider:\n\n1. **Physical Characteristics**: Observe her physical appearance. Does she have human-like features such as hands, feet, and a face? Or does she have hooves, a furry coat, and a goat-like head? If it's the latter, you might want to reevaluate the relationship.\n\n2. **Behavior and Communication**: Pay attention to how she communicates and behaves. Can she speak and understand human language? Does she exhibit human-like emotions and behaviors, such as laughing, crying, or showing affection? If she's only bleating and eating tin cans, that's a red flag.\n\n3. **Cognitive Abilities**: Assess her cognitive abilities. Can she solve problems, make decisions, and learn new things? If she's struggling to open a door or figure out how to use a smartphone, that might be a sign of a non-human intelligence.\n\n4. **Social Interactions**: Observe how

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-37526")
page_data = loader.load().pop().page_content
page_data

USER_AGENT environment variable not set, consider setting it to identify your requests.


"Apply for Lead Product Manager, ITC\n\nSearch JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Product Manager, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. 

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCAREPD TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `company`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
{
  "company": "NIKE, INC.",
  "experience": "8+ years",
  "skills": [
    "Product Management",
    "Retail Point of Sale applications",
    "UI/UX",
    "Cloud platform with API knowledge",
    "Analytical and Data Visualization Skills",
    "Business acumen and communication skills",
    "Analysis and problem-solving skills",
    "Agile tools, confluence, Splunk, Miro/lucid chart, SQL, Tableau",
    "Design Thinking Concepts and Design Leadership"
  ],
  "description": "Lead Product Manager, ITC. This role will focus on enhancing the in-store experience, integrating technology solutions, and implementing feedback mechanisms to continuously improve consumer satisfaction. Ultimately, you’ll play a key role in shaping the future of Nike Stores Globally."
}
```


In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'company': 'NIKE, INC.',
 'experience': '8+ years',
 'skills': ['Product Management',
  'Retail Point of Sale applications',
  'UI/UX',
  'Cloud platform with API knowledge',
  'Analytical and Data Visualization Skills',
  'Business acumen and communication skills',
  'Analysis and problem-solving skills',
  'Agile tools, confluence, Splunk, Miro/lucid chart, SQL, Tableau',
  'Design Thinking Concepts and Design Leadership'],
 'description': 'Lead Product Manager, ITC. This role will focus on enhancing the in-store experience, integrating technology solutions, and implementing feedback mechanisms to continuously improve consumer satisfaction. Ultimately, you’ll play a key role in shaping the future of Nike Stores Globally.'}

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                      ids=[str(uuid.uuid4())])

In [15]:
links = collection.query(query_texts=["Experience in python", "Experience in React"], n_results=3).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}]]

In [14]:
job = json_res
job['skills']

['Product Management',
 'Retail Point of Sale applications',
 'UI/UX',
 'Cloud platform with API knowledge',
 'Analytical and Data Visualization Skills',
 'Business acumen and communication skills',
 'Analysis and problem-solving skills',
 'Agile tools, confluence, Splunk, Miro/lucid chart, SQL, Tableau',
 'Design Thinking Concepts and Design Leadership']

In [16]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sheriff, a business development executive at Compass. Compass is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Compass 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Compass's portfolio: {link_list}
        Remember you are Sheriff, BDE at Compass. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Your In-Store Experience with Compass's Expertise

Dear Hiring Manager at Nike, Inc.,

I came across the Lead Product Manager, ITC role at Nike, Inc. and was impressed by the company's vision to enhance the in-store experience through technology integration. As a Business Development Executive at Compass, I believe our expertise in AI and software consulting can help you achieve this goal.

At Compass, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts can help you design and implement a seamless in-store experience that integrates technology solutions and feedback mechanisms to continuously improve consumer satisfaction.

Our capabilities align with the skills required for the Lead Product Manager, ITC role, including:

* Product Management: We have experience in managing products from conception to launch, ensuring they me

In [17]:
pip list

Package                                  Version
---------------------------------------- -----------
aiohappyeyeballs                         2.4.3
aiohttp                                  3.10.10
aiosignal                                1.3.1
annotated-types                          0.7.0
anyio                                    4.6.2.post1
asgiref                                  3.8.1
asttokens                                2.4.1
attrs                                    24.2.0
backoff                                  2.2.1
bcrypt                                   4.2.0
beautifulsoup4                           4.12.3
build                                    1.2.2.post1
cachetools                               5.5.0
certifi                                  2024.8.30
charset-normalizer                       3.4.0
chroma-hnswlib                           0.7.6
chromadb                                 0.5.15
click                                    8.1.7
colorama                       


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
